<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [2]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [3]:
# Afficher les traitements disponibles dans la pipeline fr_core_news_sm
print("Composants disponibles dans la pipeline avant désactivation :")
print(nlp.pipe_names)

# Désactiver tous les composants sauf tok2vec, morphologizer et parser
disabled_pipes = []
for pipe_name in nlp.pipe_names:
    if pipe_name not in ['tok2vec', 'morphologizer', 'parser']:
        nlp.disable_pipe(pipe_name)
        disabled_pipes.append(pipe_name)

# Vérifier que la désactivation a bien fonctionné
print("\nComposants désactivés :", disabled_pipes)
print("\nComposants actifs dans la pipeline après désactivation :")
print(nlp.pipe_names)

Composants disponibles dans la pipeline avant désactivation :
['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Composants désactivés : ['attribute_ruler', 'lemmatizer', 'ner']

Composants actifs dans la pipeline après désactivation :
['tok2vec', 'morphologizer', 'parser']


In [4]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [5]:
doc = nlp(s1)
for token in doc:
    print(f"{token.text} {token.pos_} [{token.dep_}]")

San PRON [nsubj]
Francisco PROPN [flat:name]
envisage VERB [ROOT]
d' ADP [case]
interdire NOUN [obl:arg]
les DET [det]
robots NOUN [obj]
coursiers ADJ [amod]
sur ADP [case]
les DET [det]
trottoirs NOUN [nmod]


>Dans cette représentation, l'information essentielle qui manque est l'indication du parent de chaque token dans l'arbre de dépendances (à quel mot chaque token est rattaché). Nous voyons bien le type de relation (nsubj, flat:name, ROOT, etc.) mais pas vers quel mot cette relation est dirigée, ce qui rend difficile la compréhension complète de la structure syntaxique de la phrase.

**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [6]:
# Identifier les tokens qui sont des sujets (nsubj) ou des objets (obj)
nsubj_obj_tokens = [token for token in doc if token.dep_ in ["nsubj", "obj"]]

# Afficher les sous-arbres correspondants
print("Groupes de mots qui sont des sujets ou des objets dans la phrase :")
for token in nsubj_obj_tokens:
    print(token.text, [child.text for child in token.children], token.dep_)

Groupes de mots qui sont des sujets ou des objets dans la phrase :
San ['Francisco'] nsubj
robots ['les', 'coursiers', 'trottoirs'] obj


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

In [14]:
# Les informations sur les relations de dépendance sont dans la 8ème colonne. Elles sont représentées par le nom de la relation de dépendance.
# Example : "nsubj" pour "nom sujet", "ROOT" pour le noeud racine, "det" pour "déterminant", "amod" pour "adjectif modificateur", "punct" pour "ponctuation", etc.

In [17]:
from spacy.tokens import DocBin, Doc
test_data = DocBin().from_disk("C:/Users/faxiz/OneDrive/Bureau/HEIG-VD/Semestre8/TAL/Laboratoire/L2/TAL-L2/spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [18]:
docs = list(test_data.get_docs(nlp.vocab))
if len(docs) > 1:
    doc2 = list(docs[1].sents)
    if len(doc2) > 6:
        s2 = doc2[6]
        print(s2.text)

Trois ans plus tard, il tient un discours sur la crise.


**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [19]:
from spacy import displacy
from IPython.display import HTML, display


In [20]:
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(s2, style="dep", jupyter=False)
)))

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [21]:
nlpS2 = nlp(s2.text)

display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(nlpS2, style="dep", jupyter=False)
)))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

In [28]:
# Comparer les structures de dépendances
correct_deps = 0
total_deps = 0

s2_deps = [(token.i, token.dep_, token.head.i) for token in s2]
nlpS2_deps = [(token.i, token.dep_, token.head.i) for token in nlpS2]

# Afficher les dépendances côte à côte pour comparaison
print("Original vs. Predicted")
print("=======================")
for i, (orig, pred) in enumerate(zip(s2_deps, nlpS2_deps)):
    orig_token = s2[i].text
    is_correct = orig[1] == pred[1] and (orig[2] - 161) == pred[2]
    
    if is_correct:
        correct_deps += 1
    total_deps += 1
    
    print(f"{orig_token}: [{orig[1]}] {s2[(orig[2] - 161)].text if (orig[2] - 161) < len(s2) else 'ROOT'} vs [{pred[1]}] {nlpS2[pred[2]].text if pred[2] < len(nlpS2) else 'ROOT'} {'✓' if is_correct else '✗'}")

# Calculer le taux de correction
accuracy = correct_deps / total_deps if total_deps > 0 else 0
print(f"\nTaux de correction: {accuracy:.3f} ({correct_deps}/{total_deps})")

Original vs. Predicted
Trois: [nummod] ans vs [nummod] ans ✓
ans: [obl] tient vs [obl:mod] plus ✗
plus: [advmod] tard vs [advmod] tard ✓
tard: [advmod] ans vs [ROOT] tard ✗
,: [punct] tient vs [punct] tient ✓
il: [nsubj] tient vs [nsubj] tient ✓
tient: [ROOT] tient vs [ROOT] tient ✓
un: [det] discours vs [det] discours ✓
discours: [obj] tient vs [obj] tient ✓
sur: [case] crise vs [case] crise ✓
la: [det] crise vs [det] crise ✓
crise: [nmod] discours vs [nmod] discours ✓
.: [punct] tient vs [punct] tient ✓

Taux de correction: 0.846 (11/13)


**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [29]:
from spacy.scorer import Scorer
from spacy.training import Example

In [30]:
# Créer un scorer et une exemple pour l'évaluation
scorer = Scorer()

# Créer un exemple pour l'évaluation
example = Example(nlpS2, s2.as_doc())

# Calculer les scores (scorer.score prend une liste d'exemples)
scores = scorer.score([example])

# Afficher les scores des dépendances avec trois décimales
print(f"UAS: {scores['dep_uas']:.3f}")  # Unlabeled Attachment Score
print(f"LAS: {scores['dep_las']:.3f}")  # Labeled Attachment Score

# Comparer avec les résultats de la question précédente
# La justification est imprimée pour expliquer la différence/similarité
print("\nComparaison avec le taux de correction précédent:")
print(f"Taux précédent: {correct_deps / total_deps:.3f}")
print("La différence entre les scores peut être expliquée par la méthode de comparaison.")
print("Dans la question précédente, le score inclut la comparaison de la ponctuation. Le score UAS/LAS ne le fait pas.")
print(f"Si on exlut la ponctuation, le score est le meme: {(correct_deps - 2) / (total_deps - 2):.3f}")

UAS: 0.818
LAS: 0.818

Comparaison avec le taux de correction précédent:
Taux précédent: 0.846
La différence entre les scores peut être expliquée par la méthode de comparaison.
Dans la question précédente, le score inclut la comparaison de la ponctuation. Le score UAS/LAS ne le fait pas.
Si on exlut la ponctuation, le score est le meme: 0.818


## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [34]:
# Calculate both scores for all test data
total_examples = []

# Get all docs from test data
test_docs = list(test_data.get_docs(nlp.vocab))

# Process each document and create examples for scoring
for doc in test_docs:
    for sent in doc.sents:
        # Create a Doc object from the text of the sentence for processing with nlp
        pred_doc = nlp(sent.text)
        # Create an Example object for each sentence
        example = Example(pred_doc, sent.as_doc())
        total_examples.append(example)

# Score all examples
full_scorer = Scorer()
scores = full_scorer.score(total_examples)

# Display the scores
print(f"UAS (Unlabeled Attachment Score): {scores['dep_uas']:.3f}")
print(f"LAS (Labeled Attachment Score): {scores['dep_las']:.3f}")

# Compare with the documented scores for fr_core_news_sm
print("\nDocumented scores for fr_core_news_sm:")
print("UAS (documented): 0.88")
print("LAS (documented): 0.84")
print("\nDifference from documented scores:")
print(f"UAS difference: {scores['dep_uas'] - 0.88:.3f}")
print(f"LAS difference: {scores['dep_las'] - 0.84:.3f}")

UAS (Unlabeled Attachment Score): 0.741
LAS (Labeled Attachment Score): 0.613

Documented scores for fr_core_news_sm:
UAS (documented): 0.88
LAS (documented): 0.84

Difference from documented scores:
UAS difference: -0.139
LAS difference: -0.227


**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [39]:
# Calculate scores for each dependency type
scores_by_type = scores['dep_las_per_type']

#print(scores_by_type)

# Sort the types by f score
sorted_types = sorted(scores_by_type.items(), key=lambda x: x[1]['f'], reverse=True)

# Display types with top f scores
print(f"{'Type':<20} {'Precision':>10} {'Recall':>10} {'F-score':>10}")
print("-" * 52)

# Print scores for each type, aligned in columns
for dep_type, score in sorted_types:
    print(f"{dep_type:<20} {score['p']:>10.3f} {score['r']:>10.3f} {score['f']:>10.3f}")

Type                  Precision     Recall    F-score
----------------------------------------------------
det                       0.935      0.760      0.838
cc                        0.806      0.787      0.796
case                      0.860      0.728      0.789
mark                      0.747      0.779      0.763
nsubj:pass                0.683      0.860      0.761
aux:pass                  0.636      0.925      0.754
cop                       0.729      0.734      0.731
nsubj                     0.787      0.677      0.727
root                      0.630      0.791      0.701
amod                      0.665      0.740      0.701
advmod                    0.698      0.692      0.695
nummod                    0.762      0.637      0.694
obj                       0.695      0.685      0.690
flat:name                 0.679      0.622      0.649
nmod                      0.590      0.595      0.592
xcomp                     0.496      0.613      0.549
acl:relcl                 0.5

## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [29]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [26]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
!python -m spacy train config.cfg \
  --output ./myDEPparser1 \
  --paths.train ../Labo2/spacy_data/fr-ud-train.spacy \
  --paths.dev ../Labo2/spacy_data/fr-ud-dev.spacy \
  --verbose

^C


In [ ]:
"""
Après 1h10 le score semble converger vers 0.89. Il faut environ 3 epoches pour que le score converge vers 0.88.

D'après l'analyse des données d'entraînement, il semble que le score commence à se stabiliser autour de 0.88 dès la 3ème époque. En effet, à partir de l'époque 3 (environ 5 800 itérations), les performances mesurées en termes de POS accuracy (93.49 %), de morph accuracy (93.08 %), de DEP_UAS (84.22 %) et de DEP_LAS (80.70 %) n'évoluent plus de manière significative.

Passer à 6 ou 7 époques apporte un léger gain, avec un score qui oscille entre 0.88 et 0.89, mais au prix d'un temps d'entraînement beaucoup plus long. Par exemple, à la 6ème époque (10 000 itérations), le score atteint 0.89 avec une POS accuracy de 93.66 % et une DEP_LAS de 82.07 %, soit un gain relativement faible comparé au temps nécessaire pour atteindre ce résultat.

Le compromis idéal semble donc se situer autour de 3 époques, car au-delà, l'amélioration est minime par rapport au temps investi. Cela permet d'obtenir un bon équilibre entre performance et efficacité du temps d'entraînement.
"""

"""
E    #       LOSS TOK2VEC  LOSS MORPH...  LOSS PARSER  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_F  SCORE
---  ------  ------------  -------------  -----------  -------  ---------  -------  -------  -------  ------
  0       0          0.00         224.53       504.38    34.04      29.37    24.62     9.44     0.15    0.24
  0     200       4096.78       19210.50     37619.39    87.06      82.57    71.86    63.73    77.47    0.76
  0     400       6086.66        9551.02     25690.90    90.13      87.90    76.39    70.25    86.82    0.81
  0     600       6141.56        7278.40     22205.72    91.27      89.71    77.94    72.40    92.41    0.83
  0     800       6830.94        6568.15     22035.75    91.80      90.43    79.78    74.66    92.66    0.84
  0    1000       6455.47        5613.27     19689.17    92.22      91.04    81.22    75.96    94.69    0.85
  0    1200       6854.82        5386.34     19385.73    92.45      91.52    81.36    76.63    95.68    0.85
  0    1400       7042.58        4984.85     19073.12    92.66      91.73    81.96    77.27    94.28    0.86
  1    1600       6834.45        4331.69     17409.78    92.86      91.95    82.00    77.35    94.31    0.86
  1    1800       7003.48        3973.43     16621.10    92.82      92.08    81.82    77.44    95.18    0.86
  1    2000       7270.27        4057.85     16804.29    92.93      92.26    82.64    78.30    95.09    0.87
  1    2200       7784.44        3934.81     16801.02    93.00      92.29    83.08    78.73    95.64    0.87
  1    2400       7957.06        4038.75     16813.73    93.11      92.45    83.41    79.41    95.22    0.87
  1    2600       8364.00        4107.73     17427.06    93.08      92.50    83.46    79.37    96.11    0.87
  1    2800       8355.31        4049.22     16900.29    93.22      92.75    83.92    79.85    95.92    0.87
  2    3000       8685.64        3784.67     16781.63    93.43      92.83    83.94    79.98    96.16    0.88
  2    3200       8460.56        3103.63     15042.00    93.33      92.84    83.55    79.33    95.85    0.87
  2    3400       9045.77        3369.32     15400.25    93.30      92.82    83.78    79.85    96.08    0.87
  2    3600       9386.08        3461.27     15310.16    93.33      92.88    83.89    79.88    96.02    0.87
  2    3800       9457.70        3322.98     15019.68    93.37      92.90    83.92    79.88    95.64    0.88
  2    4000       9689.28        3133.53     15105.83    93.34      92.87    83.83    80.02    95.85    0.88
  2    4200       9882.13        3246.40     15017.14    93.39      92.97    84.03    80.30    95.69    0.88
  3    4400       9699.11        3182.83     14413.34    93.38      92.99    84.34    80.36    96.03    0.88
  3    4600      10288.52        2935.46     14225.83    93.36      92.99    84.42    80.70    96.30    0.88
  3    4800      10484.30        2823.29     13812.97    93.44      93.04    84.14    80.37    95.92    0.88
  3    5000      11164.27        2900.67     14236.61    93.37      93.03    83.79    80.24    95.32    0.88
  3    5200      11474.90        2883.83     14451.45    93.39      92.98    84.35    80.53    95.95    0.88
  3    5400      11381.88        2826.25     14024.44    93.49      93.08    84.22    80.70    96.01    0.88
  3    5600      11518.92        2921.21     13766.86    93.52      93.10    84.48    80.75    95.51    0.88
  3    5800      11825.22        2959.30     13940.46    93.61      93.10    84.87    81.01    96.13    0.88
  4    6000      11401.72        2456.21     12922.77    93.58      93.15    84.87    81.29    95.83    0.88
  4    6200      12716.10        2614.73     13143.19    93.51      93.06    84.84    81.06    96.36    0.88
  4    6400      12803.21        2547.68     12967.38    93.51      93.16    84.79    81.07    95.34    0.88
  4    6600      13767.46        2677.54     13708.22    93.59      93.19    84.53    81.06    96.06    0.88
  4    6800      13726.32        2587.43     13400.85    93.64      93.30    84.51    80.94    96.09    0.88
  4    7000      13754.91        2721.75     13463.51    93.65      93.28    84.73    81.17    96.46    0.88
  4    7200      13792.97        2600.69     13299.43    93.61      93.17    84.86    81.30    95.85    0.88
  5    7400      13637.91        2553.52     12850.60    93.55      93.20    85.10    81.55    96.47    0.88
  5    7600      14250.57        2280.12     12321.21    93.51      93.19    84.70    81.20    95.77    0.88
  5    7800      15413.42        2403.21     12570.02    93.62      93.25    84.85    81.33    95.71    0.88
  5    8000      15926.62        2447.42     12804.97    93.45      93.10    84.75    81.37    96.36    0.88
  5    8200      16224.26        2482.72     13298.59    93.52      93.19    84.70    81.13    96.33    0.88
  5    8400      15862.43        2494.86     12387.48    93.62      93.21    84.80    81.50    96.26    0.88
  5    8600      15555.87        2345.07     12120.99    93.63      93.27    84.66    81.26    96.73    0.88
  6    8800      16566.15        2287.05     12804.26    93.65      93.32    84.94    81.50    96.06    0.88
  6    9000      16152.99        2148.36     11473.04    93.67      93.27    85.33    81.83    96.70    0.89
  6    9200      17916.59        2261.99     11944.51    93.64      93.19    84.88    81.50    96.06    0.88
  6    9400      18085.37        2131.71     11770.50    93.64      93.24    85.16    81.70    96.29    0.88
  6    9600      18234.57        2261.31     11918.78    93.62      93.21    85.04    81.69    96.08    0.88
  6    9800      18905.74        2350.16     12344.38    93.62      93.25    85.28    81.77    96.20    0.88
  6   10000      19928.89        2436.91     12446.10    93.66      93.27    85.44    82.07    96.02    0.89
  7   10200      18905.90        2292.02     11893.45    93.62      93.17    85.31    81.91    96.15    0.89
  7   10400      18197.61        1898.43     10801.62    93.58      93.26    84.72    81.52    96.29    0.88
  7   10600      20465.24        2118.91     11420.77    93.58      93.25    85.37    81.87    96.29    0.89
"""

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [42]:
# Charger le modèle entraîné
nlp2 = spacy.load("./myDEPparser1/model-best")

# Calculate both scores for all test data
total_examples = []

# Get all docs from test data
test_docs = list(test_data.get_docs(nlp2.vocab))

# Process each document and create examples for scoring
for doc in test_docs:
    for sent in doc.sents:
        # Create a Doc object from the text of the sentence for processing with nlp2
        pred_doc = nlp2(sent.text)
        # Create an Example object for each sentence
        example = Example(pred_doc, sent.as_doc())
        total_examples.append(example)

# Score all examples
scores = full_scorer.score(total_examples)

# Display the scores
print(f"UAS (Unlabeled Attachment Score): {scores['dep_uas']:.3f}")
print(f"LAS (Labeled Attachment Score): {scores['dep_las']:.3f}")

# Compare with the scores from fr_core_news_sm
print("\nComparison with fr_core_news_sm scores:")
print(f"UAS difference: {scores['dep_uas'] - 0.741:.3f}")  
print(f"LAS difference: {scores['dep_las'] - 0.613:.3f}")

"""
Le modèle entraîné (nlp2) a obtenu les scores suivants sur les données de test :
- UAS (Unlabeled Attachment Score) : 0.687
- LAS (Labeled Attachment Score) : 0.597

En comparaison, le modèle de base fr_core_news_sm avait obtenu :
- UAS : 0.741
- LAS : 0.613

L’analyse des résultats montre que notre modèle personnalisé a des performances légèrement inférieures à fr_core_news_sm. En particulier :

- Le UAS est inférieur de 0.054, indiquant que la structure des dépendances est moins bien capturée.

- Le LAS est inférieur de 0.016, ce qui signifie que l’étiquetage des relations reste relativement proche du modèle pré-entraîné.

Cela suggère que notre modèle a encore une marge d’amélioration. Pour obtenir de meilleurs résultats, il serait pertinent d’augmenter le nombre d’itérations (epoch), d’enrichir les données d’entraînement ou d’utiliser un modèle de base plus performant comme fr_core_news_md ou fr_core_news_lg.
"""

UAS (Unlabeled Attachment Score): 0.687
LAS (Labeled Attachment Score): 0.597

Comparison with fr_core_news_sm scores:
UAS difference: -0.054
LAS difference: -0.016


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [43]:
# Calculate scores for each dependency type
scores_by_type = scores['dep_las_per_type']

# Sort the types by f score
sorted_types = sorted(scores_by_type.items(), key=lambda x: x[1]['f'], reverse=True)

# Display types with top f scores
print(f"{'Type':<20} {'Precision':>10} {'Recall':>10} {'F-score':>10}")
print("-" * 52)

# Print scores for each type, aligned in columns
for dep_type, score in sorted_types:
    print(f"{dep_type:<20} {score['p']:>10.3f} {score['r']:>10.3f} {score['f']:>10.3f}")

Type                  Precision     Recall    F-score
----------------------------------------------------
det                       0.901      0.768      0.829
case                      0.784      0.728      0.755
cc                        0.766      0.709      0.736
aux                       0.692      0.769      0.728
flat:name                 0.662      0.723      0.691
aux:pass                  0.615      0.755      0.678
nummod                    0.611      0.699      0.652
amod                      0.628      0.659      0.643
nsubj                     0.681      0.594      0.635
cop                       0.690      0.576      0.627
nmod                      0.554      0.675      0.609
root                      0.484      0.663      0.560
obj                       0.491      0.632      0.552
advmod                    0.593      0.515      0.551
mark                      0.684      0.450      0.543
obl                       0.448      0.438      0.443
xcomp                     0.3

**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

In [ ]:
"""
En comparant les scores détaillés avant et après l'entraînement du modèle, on observe plusieurs changements notables :

En haut du classement (Top 3 labels) :
- "det" (déterminant) :
    - Avant : F-score = 0.838
    - Après : F-score = 0.829
  Légère baisse, mais le score reste élevé.

- "case" (mot de relation, ex: prépositions) :
    - Avant : F-score = 0.789
    - Après : F-score = 0.755
  Baisse de 0.034, indiquant une performance légèrement dégradée.

- "cc" (conjonction de coordination) :
    - Avant : F-score = 0.796
    - Après : F-score = 0.736
  Perte de 0.060, ce qui montre une dégradation notable.


En bas du classement :
- Plusieurs labels restent à 0.000 avant et après l'entraînement, comme "ccomp" (complément de phrase), "parataxis" (juxtaposition), "iobj" (objet indirect), "obj:agent", "aux:caus", etc.

- Cela signifie que le modèle n'a pas réussi à améliorer la reconnaissance de ces relations, ce qui pourrait être dû à un manque d'exemples d'entraînement pour ces catégories.


Labels qui n'ont pas progressé :
- "expl" (expletive) :
    - Avant : 0.000
    - Après : F-score = 0.164
  Une petite amélioration, mais le score reste faible.

- "obl" (complément oblique) :
    - Avant : 0.000
    - Après : F-score = 0.443
  Une nette amélioration, mais cela reste un score moyen.

Conclusion :
L'entraînement du modèle a globalement dégradé la performance sur les relations les plus fréquentes (comme det, case, cc), tandis que certaines relations plus rares ont légèrement progressé. Cependant, plusieurs labels restent non reconnus. Pour améliorer ces résultats, il faudrait probablement :

    - Augmenter le nombre d’itérations (epochs) pour mieux ajuster le modèle.
    - Ajouter plus de données d'entraînement, notamment pour les relations sous-représentées.
    - Tester avec un modèle de base plus grand (fr_core_news_md ou fr_core_news_lg).
"""

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.